In [140]:
import findspark
#inicializacao spark
findspark.init()
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import quinn
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
#Biblioteca para conexao com o PostgreSQL
import psycopg2

In [157]:
#Conexao com o Spark
sqlCtx = SQLContext(spark)
data = sqlCtx.read.options(header='False').csv("/home/hugosouza/Documentos/haberman.data")
print(data)
data.show(10)

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string]
+---+---+---+---+
|_c0|_c1|_c2|_c3|
+---+---+---+---+
| 30| 64|  1|  1|
| 30| 62|  3|  1|
| 30| 65|  0|  1|
| 31| 59|  2|  1|
| 31| 65|  4|  1|
| 33| 58| 10|  1|
| 33| 60|  0|  1|
| 34| 59|  0|  2|
| 34| 66|  9|  2|
| 34| 58| 30|  1|
+---+---+---+---+
only showing top 10 rows



In [159]:
#renomear as colunas com o cabeçalho ausente
dataM = data.withColumnRenamed('_c0','IdadeOperado')\
            .withColumnRenamed('_c1','AnoOperacao')\
            .withColumnRenamed('_c2','QntNodulos')\
            .withColumnRenamed('_c3','Status')
dataM.show(10)

+------------+-----------+----------+------+
|IdadeOperado|AnoOperacao|QntNodulos|Status|
+------------+-----------+----------+------+
|          30|         64|         1|     1|
|          30|         62|         3|     1|
|          30|         65|         0|     1|
|          31|         59|         2|     1|
|          31|         65|         4|     1|
|          33|         58|        10|     1|
|          33|         60|         0|     1|
|          34|         59|         0|     2|
|          34|         66|         9|     2|
|          34|         58|        30|     1|
+------------+-----------+----------+------+
only showing top 10 rows



In [164]:
dataMortes = dataM[dataM['Status']=='1']
dataMortes.show(10)

+------------+-----------+----------+------+
|IdadeOperado|AnoOperacao|QntNodulos|Status|
+------------+-----------+----------+------+
|          30|         64|         1|     1|
|          30|         62|         3|     1|
|          30|         65|         0|     1|
|          31|         59|         2|     1|
|          31|         65|         4|     1|
|          33|         58|        10|     1|
|          33|         60|         0|     1|
|          34|         58|        30|     1|
|          34|         60|         1|     1|
|          34|         61|        10|     1|
+------------+-----------+----------+------+
only showing top 10 rows



In [179]:
#conexao postgresql
conn = psycopg2.connect(host="localhost", database="postgres",
                        user = "postgres",
                        password = "------")

In [180]:
cur = conn.cursor()

In [181]:
#criar tabelas no bd
cur.execute("""CREATE TABLE sobrevida 
           ( IdadeOperado integer, 
             AnoOperacao integer, 
             QntNodulos integer,
             Status integer);""")

In [182]:
a = [tuple(x) for x in dataMortes.collect()]

In [183]:
b = ','.join(['%s'] * len(a))

In [184]:
#insercao dos dados
q = "INSERT INTO sobrevida (IdadeOperado,AnoOperacao,QntNodulos,Status) VALUES {}".format(b)

In [185]:
cur.execute(q, a)

In [186]:
cur.execute("SELECT * FROM mortes2;")

In [187]:
result = cur.fetchmany(5)
for row in result:
    print(row)
conn.commit()

(30, 64, 1, 1)
(30, 62, 3, 1)
(30, 65, 0, 1)
(31, 59, 2, 1)
(31, 65, 4, 1)


In [188]:
#encerrar conexao
cur.close()
conn.close()